In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install nflows --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pylab import savefig
from scipy.io import arff
import ntpath
import glob
import os
import math
from sklearn import preprocessing
# !pip install liac-arff
#import arff
import argparse

import torch
from torch import nn, optim
from nflows.flows import Flow
from nflows.distributions import StandardNormal
from nflows.transforms import CompositeTransform, MaskedAffineAutoregressiveTransform, RandomPermutation
from nflows.transforms import AffineCouplingTransform

from sklearn import manifold
import string


In [ ]:
import pickle
# Load object from file
with open(gdrivePath +os.sep + "data"+os.sep+'X_train.pkl', 'rb') as f:
    X_train = pickle.load(f)
with open(gdrivePath +os.sep + "data" +os.sep+'X_test.pkl', 'rb') as f:
    X_test = pickle.load(f)
with open(gdrivePath +os.sep + "data"+os.sep+'y_train.pkl', 'rb') as f:
    y_train = pickle.load(f)
with open(gdrivePath +os.sep + "data"+os.sep+'y_test.pkl', 'rb') as f:
    y_test = pickle.load(f)


In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
unique_values, counts = np.unique(y_train, return_counts=True)
display(dict(zip(unique_values, counts)),np.max(counts))

In [ ]:
X_train[0].shape

In [ ]:
y_train

array(['ductal', 'acinar', 'delta', ..., 'beta', 'beta', 'beta'],
      dtype=object)

In [ ]:
from sklearn.preprocessing import OneHotEncoder


# Reshape the array to 2D (required for OneHotEncoder)
y_reshaped = y_train.reshape(-1, 1)

# Initialize the OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# Fit and transform the data
y_one_hot_ori = encoder.fit_transform(y_reshaped)

# Print the one-hot encoded array
print("One-Hot Encoded Array:")
print(y_one_hot_ori)

# Optionally, print the categories
print("Categories:")
print(encoder.categories_)


One-Hot Encoded Array:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Categories:
[array(['PSC', 'acinar', 'activated_stellate', 'alpha', 'beta', 'delta',
       'ductal', 'endothelial', 'epsilon', 'gamma', 'macrophage', 'mast',
       'mesenchymal', 'pp', 'quiescent_stellate', 'schwann'], dtype=object)]


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Step 1: Encode the categories as integers
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_train)  # Convert categories to integers

# Step 2: Convert the encoded labels to a torch.LongTensor
y_tensor = torch.tensor(y_encoded, dtype=torch.long)

In [ ]:
y_tensor

tensor([6, 1, 5,  ..., 4, 4, 4])

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
import torch
import torch.optim as optim
import numpy as np
from nflows.distributions import StandardNormal
from nflows.transforms import MaskedAffineAutoregressiveTransform, ReversePermutation, CompositeTransform
from nflows.flows import Flow


# Assuming X_moons and label_moon are already defined
# `label_moon` contains class labels (e.g., 0, 1, 2, ..., n_classes - 1)
n_features = X_train.shape[1]
n_layers = 1
n_classes = len(np.unique(y_tensor))  # Determine the number of classes

# Base distribution
base_dist = StandardNormal(shape=[n_features])

# Define the transform
transforms = []
for i in range(n_layers):
    transforms.append(
        MaskedAffineAutoregressiveTransform(
            features=n_features,
            hidden_features=1024,
            context_features=n_classes  # Updated for multi-class
        )
    )
    transforms.append(ReversePermutation(features=n_features))

transform = CompositeTransform(transforms)

# Define the flow
flow = Flow(transform, base_dist).to(device)

# Optimizer
optimizer = optim.Adam(flow.parameters(),lr=1e-6)

# Training parameters
num_epochs = 50
batch_size = 128
max_batches = int(X_train.shape[0] / batch_size)

# Training loop
for epoch in range(num_epochs):
    permut = np.random.permutation(X_train.shape[0])
    X_moons_shuffle = X_train[permut]
    label_moon_shuffle = y_tensor[permut]

    for i_batch in range(max_batches):
        # Get the batch
        x = X_moons_shuffle[i_batch * batch_size:(i_batch + 1) * batch_size]
        x = torch.tensor(x,device=device).float()

        y = label_moon_shuffle[i_batch * batch_size:(i_batch + 1) * batch_size]
        y_one_hot = torch.nn.functional.one_hot(
            torch.tensor(y,device=device), num_classes=n_classes
        ).float()  # One-hot encoding

        # Zero gradients
        optimizer.zero_grad()

        # Compute negative log likelihood
        nll = -flow.log_prob(x, context=y_one_hot)  # Use one-hot encoded context
        loss = nll.mean()

        # Backpropagation
        loss.backward()
        optimizer.step()

    # Logging
    if epoch % 10 == 0:
        print(f"Epoch: {epoch}, Loss: {loss.item():.4f}")


In [ ]:
unique_values, counts = np.unique(y_train, return_counts=True)
#display(dict(zip(unique_values, counts)),np.max(counts))
counts

array([  42, 1090,  227, 3897, 2950,  759, 1360,  231,   17,  339,   44,
         20,   64,  148,  138,   11])

In [ ]:
threshold = 759

# Calculate samples_per_class
#samples_per_class = np.maximum(threshold - counts, 0)

# Threshold value
threshold = 759

# Create a dictionary for samples_per_class
samples_per_class = {}

# Loop through the counts and calculate the number of samples to generate
for class_id, count in enumerate(counts):
    # If count is less than the threshold, calculate how many samples to generate
    if count < threshold:
        samples_per_class[class_id] = threshold - count
    else:
        samples_per_class[class_id] = 0  # No samples needed for this class

# Print the result
print("Samples Per Class:", samples_per_class)


Samples Per Class: {0: 717, 1: 0, 2: 532, 3: 0, 4: 0, 5: 0, 6: 0, 7: 528, 8: 742, 9: 420, 10: 715, 11: 739, 12: 695, 13: 611, 14: 621, 15: 748}


In [ ]:
# Create the context tensor
cond_list = []
for class_id, num_samples in samples_per_class.items():
    # Create one-hot encoded context for the class
    class_context = torch.nn.functional.one_hot(
        torch.tensor([class_id] * num_samples,dtype=torch.long), num_classes=n_classes
    )
    cond_list.append(class_context)

# Concatenate all contexts
cond = torch.cat(cond_list, dim=0).float().to(device)

# Generate samples using the flow model
with torch.no_grad():
    samples = flow.sample(1, context=cond).view(-1, n_features).cpu().numpy()

# Print the shape of the generated samples
#print(samples.shape)  # Should match the total number of samples (e.g., 100 + 50 + 150 = 300)


In [ ]:
combined = np.vstack((X_train, samples))

In [ ]:
y_one_hot_ori.shape

(11337, 16)

In [ ]:
cond_npy = cond.cpu().numpy()
cond_npy

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [ ]:
cond_npy.shape

(7068, 16)

In [ ]:
combined_y = np.vstack((y_one_hot_ori, cond_npy))
combined_y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [ ]:
print(combined.shape, combined_y.shape)

(18405, 7514) (18405, 16)


In [ ]:
y_train_indexes = np.full(len(y_one_hot_ori), 1)
y_train_indexes = np.concatenate([y_train_indexes, np.full(len(samples), 2)])

In [ ]:
y_train_indexes.shape

(18405,)

In [ ]:
file_name ="FB"

In [ ]:
import pickle

# Save object to file
with open(gdrivePath +os.sep+"results"+os.sep+"FB"+os.sep+file_name+'_X_train.pkl', 'wb') as f:
    pickle.dump(combined, f)

with open(gdrivePath +os.sep+"results"+os.sep+"FB"+os.sep+file_name+'_y_train.pkl', 'wb') as f:
    pickle.dump(combined_y, f)

with open(gdrivePath +os.sep+"results"+os.sep+"FB"+os.sep+file_name+'_y_train_idx.pkl', 'wb') as f:
    pickle.dump(y_train_indexes, f)